# Code for analysis of GM to WM contrast in SC images and SC QSM processing steps

In [10]:
import nibabel as nib
import numpy as np  
import sys
import os

In [2]:
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy import stats

In [5]:
import importlib
from algo_comp_from_folder import calculate_t_statistic_between_gm_wm

In [4]:
# Lets begin analyzing our best looking magnitude
hc2_m1_mag_img = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2_m1_mag.nii.gz")
hc2_m1_mag_data = hc2_m1_mag_img.get_fdata()
# Now lets load the custom masks
hc2_m1_gm_msk_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1/custom_hc2_m1_gm_msk.nii.gz").get_fdata()
hc2_m1_wm_msk_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_wm_msk.nii.gz").get_fdata()

### Lets calculate SNR of one of the in-vivo meGRE images

In [1]:
from utils.snr_calc import snr_calc
hc2_m1_mag_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\hc2_m1_mag.nii.gz"
signal_mask_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom1_sc_msk.nii.gz" # Using the custom SC mask
noise_mask_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\noisy_air_mask.nii.gz"
snr_hc2_m1 = snr_calc(hc2_m1_mag_path, signal_mask_path, noise_mask_path)
print("SNR for HC2 M1 on using a spinal cord mask is: ", snr_hc2_m1)

SNR for HC2 M1 on using a spinal cord mask is:  57.85094811418218


In [ ]:
# Same SNR calculation for more datasets:


# Continue to in-vivo SC QSM analysis

In [7]:
hc2_m1_opt_pdf_opt_tkd_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\chi_map\pdf_to_tkd\opt_to_opt/Sepia_Chimap.nii.gz").get_fdata()

In [5]:
nSlices = hc2_m1_mag_img.shape[2]
nSlices

16

In [6]:
contrast_values = np.full(nSlices, np.nan)

for s in range(nSlices):
    wm_slice = hc2_m1_wm_msk_data[:, :, s].astype(bool)
    gm_slice = hc2_m1_gm_msk_data[:, :, s].astype(bool)

    wm_vals = hc2_m1_mag_data[:, :, s][wm_slice]
    gm_vals = hc2_m1_mag_data[:, :, s][gm_slice]

    if wm_vals.size > 0 and gm_vals.size > 0:
        mu_wm = wm_vals.mean()
        mu_gm = gm_vals.mean()

        contrast_values[s] = (mu_gm - mu_wm) / (mu_gm + mu_wm)  # signed contrast

# contrast_values now contains the GM–WM contrast for each slice
print(contrast_values)


[       nan        nan 0.1012321  0.03194563 0.05406769 0.08772266
 0.06536509 0.0661796  0.07663165 0.07719194 0.06496995 0.08389183
 0.06915417        nan        nan        nan]


# <span style="color:orange"> Difference of mean in Chimaps // *For SC QSM* </br>
In this section we'll perform a statistical test looking to see if the difference between of the GM and WM mean is significant or not.


In [8]:
# Load the chimap you want to analyuze
# chimap_img = nib.load()
chimap_data = hc2_m1_opt_pdf_opt_tkd_data

hc1m1_chimap_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\chi_map\pdf_to_tkd/opt_to_opt\Sepia_Chimap.nii.gz"
wm_hc1m1_mask_path =r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\custom_hc1_m1_gm_msk.nii.gz"
gm_hc1m1_mask_path = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc1\m1\custom_hc1_m1_wm_msk.nii.gz"

# Now load the masks or custom masks you want to use to calculate the mean 
wm_mask_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_gm_msk.nii.gz").get_fdata()
gm_mask_data = nib.load(r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing\hc2\m1\custom_hc2_m1_wm_msk.nii.gz").get_fdata()
                      

In [9]:
t_stat_hc1m1, p_val_hc1m1 = calculate_t_statistic_between_gm_wm(hc1m1_chimap_path, wm_hc1m1_mask_path, gm_hc1m1_mask_path)

T-statistic: -14.493824, p-value: 8.690809e-41


In [23]:
# Extract voxel values
wm_vals = chimap_data[wm_mask_data == 1]
gm_vals = chimap_data[gm_mask_data == 1]

# Remove NaN/inf values
wm_vals = wm_vals[np.isfinite(wm_vals)]
gm_vals = gm_vals[np.isfinite(gm_vals)]

# Means
wm_mean = np.mean(wm_vals)
gm_mean = np.mean(gm_vals)
mean_diff = gm_mean - wm_mean  # Substract WM because it should always be negative

In [30]:
print(f"WM Mean: {wm_mean:.4f}, GM Mean: {gm_mean:.4f}, Mean Difference (GM - WM): {mean_diff:.4f}")
print("WM voxel count:", wm_vals.size)
print("GM voxel count:", gm_vals.size)
print("WM variance:", np.var(wm_vals))
print("GM variance:", np.var(gm_vals))

WM Mean: 0.0106, GM Mean: -0.0021, Mean Difference (GM - WM): -0.0128
WM voxel count: 477
GM voxel count: 2252
WM variance: 0.00014923495286389455
GM variance: 0.00011575811589836249


Not a big difference in voxel counts indicates that there may not be big imbalance (and if there is, this may not be why)

Nonzero viarance is good, also variance is similar which indicates that independendent t-test is a good fit :)

In [27]:
# Check no Nans, because this messes up the stats
print("NaNs in WM:", np.isnan(wm_vals).sum())
print("NaNs in GM:", np.isnan(gm_vals).sum())
print("Infs in WM:", np.isinf(wm_vals).sum())
print("Infs in GM:", np.isinf(gm_vals).sum())


NaNs in WM: 0
NaNs in GM: 0
Infs in WM: 0
Infs in GM: 0


In [15]:
# Just in case remove NaN/inf values again
wm_vals_clean = wm_vals[np.isfinite(wm_vals)]
gm_vals_clean = gm_vals[np.isfinite(gm_vals)]

In [28]:
t_stat, p_val = ttest_ind(wm_vals, gm_vals, equal_var=False)
print(f"T-statistic: {t_stat:.6f}, p-value: {p_val:.6e}")

T-statistic: 21.178254, p-value: 7.020576e-76


### Just in case you want to do the statistical test manually

In [22]:
# Rememmber that s^2 is the sample variance we calculated with np.var()
# means and SDs
mean_gm = 0.0088
mean_wm = -0.0019
sd_gm   = 0.012
sd_wm   = 0.012
n       = 2  # number of subjects

# standard error of difference
se_diff = np.sqrt(sd_gm**2/n + sd_wm**2/n)

# t-statistic
t = (mean_gm - mean_wm) / se_diff

# degrees of freedom (Welch-Satterthwaite)
df = (sd_gm**2/n + sd_wm**2/n)**2 / ((sd_gm**2/n)**2/(n-1) + (sd_wm**2/n)**2/(n-1))

# two-tailed p-value
p = 2 * (1 - stats.t.cdf(np.abs(t), df))

print("t =", t)
print("p =", p)


t = 0.8916666666666667
p = 0.4666574532668242
